In [1]:
import sqlite3
import pandas as pd
import quandl

# Set your Quandl API key
quandl.ApiConfig.api_key = 'FMezMmR86K7axszB_rkz'

# Define the series codes
series_codes = {"YIELD":'YIELD',"REALYIELD":'REALYIELD'}

# Prefix to be added to each series code
prefix = "USTREASURY/"

# Frequencies to download (daily, weekly, monthly)
frequencies = ['daily', 'weekly', 'monthly']

# Create an SQLite connection and cursor
conn = sqlite3.connect('Treasury_Yields_data.db')
cursor = conn.cursor()

# Fetch data and create tables for each frequency
for series_code in series_codes:
    for frequency in frequencies:
        full_series_code = prefix + series_code

        # Adjust the frequency in the Quandl request
        data = quandl.get(full_series_code, collapse=frequency)

        # Create a table for the original series
        table_name = f"{series_codes[series_code]}_{frequency}"
        # Create a table for the original series
        data.to_sql(table_name, conn, if_exists='replace', index=True)

        # Create a table for the period-to-period difference
        diff_data = data.diff()
        diff_table_name = f"{table_name}_diff"
        diff_data.to_sql(diff_table_name, conn, if_exists='replace', index=True)

        # Create a table for the percent change
        percent_change_data = data.pct_change()
        percent_change_table_name = f"{table_name}_percent_change"
        percent_change_data.to_sql(percent_change_table_name, conn, if_exists='replace', index=True)

# Commit changes and close connection
conn.commit()
conn.close()

print("Databases created successfully.")


Databases created successfully.
